In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Jul 27 02:45:16 2020

@author: joygu
"""
import pickle
import numpy as np
import csv
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict

from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
# change the file locatio here
train_path = "data/pickle/train/"
test_path = "data/pickle/test/"
#load train data
# need reshap
chroma = pickle.load( open( train_path + "chroma_stft.p", "rb" ) )
mfcc = pickle.load( open( train_path + "mfcc.p", "rb" ) )
#not need
spectral_centroid = pickle.load( open( train_path + "spectral_centroid.p", "rb" ) )
spectral_rolloff = pickle.load( open( train_path + "spectral_rolloff.p", "rb" ) )
zcr = pickle.load( open( train_path + "zcr.p", "rb" ) )

#load test data
# need reshap
chroma_test = pickle.load( open( test_path + "chroma_stft.p", "rb" ) )
mfcc_test = pickle.load( open( test_path + "mfcc.p", "rb" ) )
#not need
spectral_centroid_test = pickle.load( open( test_path + "spectral_centroid.p", "rb" ) )
spectral_rolloff_test = pickle.load( open( test_path + "spectral_rolloff.p", "rb" ) )
zcr_test = pickle.load( open( test_path + "zcr.p", "rb" ) )


# find data point used in this small data set
list_key=list(zcr.keys())
list_key_test=list(zcr_test.keys())

#%% define function
def preprocess_data(list_key, feature):
    cleaned_feature_dict = defaultdict(np.array)
    cleaned_feature=[]
    for path in list_key:
        feature_result = dict(feature)[path]
        _, n_frames = feature_result.shape
        if n_frames < 2500: ## Remove all shorter mp3 files from analysis
            continue
        feature_result_fixed = feature_result[:, :2582] ## Resizes all np.arrays to 2582 frames
        scale = StandardScaler()
        scaled_feature_result = scale.fit_transform(feature_result_fixed)
        feature_result_long = scaled_feature_result.reshape(1,-1)
        cleaned_feature_dict[path] = feature_result_long
        cleaned_feature.append(feature_result_long)
    new_list_key = dict(cleaned_feature_dict).keys()
# return new data path and dictionary of its corresponding value
    return list(new_list_key), cleaned_feature_dict,cleaned_feature

def nonmatrix_data(list_key, feature):
    cleaned_feature=[]
    for path in list_key:
        feature_result = dict(feature)[path]
        feature_result_fixed = np.array(feature_result[:, :2582]).reshape(-1,1) ## Resizes all np.arrays to 2582 frames
        scale = StandardScaler()
        scaled_feature_result = scale.fit_transform(feature_result_fixed)
        cleaned_feature.append(scaled_feature_result.reshape(1,-1))
# return new data path and dictionary of its corresponding value
    return cleaned_feature

def combinefeature(datalen,arr1,arr2,arr3,arr4,arr5):
    overallfeature=[]
    for x in range(datalen):
        a,b,c,d,e = arr1[x],arr2[x],arr3[x],arr4[x],arr5[x]
        overallfeature.append(np.concatenate((a,b,c,d,e), axis=1))
    return overallfeature

def combinefeature1(datalen,arr1,arr2,arr3,arr4,arr5):
    overallfeature=[]
    for x in range(datalen):
        a,b,c,d,e = arr1[x],arr2[x],arr3[x],arr4[x],arr5[x]
        a= np.array(arr1[x]).reshape(-1,1)[:10000].reshape(1,-1)
        b=np.array(arr2[x]).reshape(-1,1)[:10000].reshape(1,-1)
        c=np.array(arr3[x]).reshape(-1,1)[:2582].reshape(1,-1)
        d=np.array(arr4[x]).reshape(-1,1)[:2582].reshape(1,-1)
        e=np.array(arr5[x]).reshape(-1,1)[:2582].reshape(1,-1)
        overallfeature.append(np.concatenate((a,b,e), axis=1))
    return overallfeature


In [2]:
# %% data preprocessing
list_key,_, chromafeature = preprocess_data(list_key, chroma)
list_key,_, mfccfeature = preprocess_data(list_key, mfcc)
sc_feature = nonmatrix_data(list_key, spectral_centroid)
sr_feature = nonmatrix_data(list_key, spectral_rolloff)
zcr_feature = nonmatrix_data(list_key, zcr)
ttl_feature  = combinefeature(len(list_key),chromafeature,mfccfeature,sc_feature,sr_feature,zcr_feature)
#ttl_feature  = combinefeature1(len(list_key),chromafeature,mfccfeature,zcr_feature)

list_key_test,_, chromafeature_test = preprocess_data(list_key_test, chroma_test)
list_key_test,_, mfccfeature_test = preprocess_data(list_key_test, mfcc_test)
sc_feature_test = nonmatrix_data(list_key_test, spectral_centroid_test)
sr_feature_test = nonmatrix_data(list_key_test, spectral_rolloff_test)
zcr_feature_test = nonmatrix_data(list_key_test, zcr_test)
ttl_feature_test  = combinefeature(len(list_key_test),chromafeature_test,mfccfeature_test,sc_feature_test,sr_feature_test,zcr_feature_test)
#ttl_feature_test  = combinefeature1(len(list_key_test),chromafeature_test,mfccfeature_test,zcr_feature_test)

train_genere_id = [float(list_key[z].split('/')[-1][:-4]) for z in range(len(list_key))]
test_genere_id = [float(list_key_test[z].split('/')[-1][:-4]) for z in range(len(list_key_test))]
total_genere_id =[]
total_genere_id.extend(train_genere_id)
total_genere_id.extend(test_genere_id)
#train_feature = list(feature.values())

In [ ]:
# %% data preprocessing 2
TRACKS_PATH = '/Users/joygu/Documents/summer2020/isye6740/project/music-genre-classification-master/data/fma_metadata/tracks.csv'
tracks = []
trackid_t=[]
heads2=[]
i=1
rawdata = []
with open(TRACKS_PATH) as csvfile:
    readCSV1 = csv.reader(csvfile, delimiter=',')
    for row in readCSV1:
        if i<4:
            readhead=row
            heads2.append(readhead)
        else:    
            readentry = [x for x in row]#[float(x) for x in row]
            rawdata.append(readentry)
            trackid_t.append(float(readentry[0]))
            tracks.append(readentry[40])
        i=i+1
index=np.searchsorted(trackid_t,total_genere_id)
tracks_new=[tracks[index[small]] for small in range(len(index))]
 # label each catagory with numbers
le = LabelEncoder()
genre_id = le.fit_transform(tracks_new)
classes=list(le.classes_)
# fit the classes to this small data set
y_train, y_test = genre_id[:len(train_genere_id)],genre_id[len(train_genere_id):-1]
y_test=np.append(y_test,genre_id[-1])


x_train = np.array(ttl_feature).reshape(6384,-1)
x_test = np.array(ttl_feature_test).reshape(1596,-1)
print('training start')

In [ ]:
err = []
for Model in [GaussianNB, KNeighborsClassifier,MLPClassifier,LogisticRegression,SVC]:
#for Model in [MLPClassifier]:
    print(Model)
    #gaussian 0.34~ knn 0.45~ logistic default converge error? SVC 0.4775~ NN 0.455
    if Model == GaussianNB:
        clf =  GaussianNB(var_smoothing=1.29e-10)
    elif Model == LogisticRegression:
        #clf =  LogisticRegression(solver='liblinear')
        clf =  LogisticRegression(solver='liblinear',max_iter=1000)#max_iter=5000)
    elif Model == KNeighborsClassifier:
        clf = Model(n_neighbors=21)
    elif Model == MLPClassifier:
        clf =  Model( activation='relu', max_iter=1000, hidden_layer_sizes=(140,2), random_state=1)
    elif Model == SVC:
        clf = Model(kernel = 'rbf',max_iter=2000)
        #clf = Model(kernel='linear', C=1.0,max_iter=1000)
    clf.fit(x_train, y_train)
    y_pred = np.array(clf.predict(x_test),dtype=np.int)
    y_expected = np.array(y_test,dtype=np.int)
    print(np.sum(y_pred==y_expected)/y_expected.shape[0])
    err.append(np.sum(y_pred==y_expected)/y_expected.shape[0])
